# Exploration of NY Times COVID-19 data for LA County

Data from The New York Times, based on reports from state and local health agencies.
The Times is reporting at https://www.nytimes.com/interactive/2020/us/coronavirus-us-cases.html.

**Remember to `git pull upstream master` every day.** Data includes up to yesterday's total.

In [1]:
import pandas as pd
import numpy as np

import bokeh.plotting
import bokeh.models
import bokeh.io

import colorcet

import tqdm

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
%load_ext blackcellmagic

In [3]:
yesterday = pd.to_datetime(pd.to_datetime("today").date()) - pd.DateOffset(days=1)

## Load data for US counties

In [4]:
df_counties = pd.read_csv("us-counties.csv")

df_counties.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0
1,2020-01-22,Snohomish,Washington,53061.0,1,0
2,2020-01-23,Snohomish,Washington,53061.0,1,0
3,2020-01-24,Cook,Illinois,17031.0,1,0
4,2020-01-24,Snohomish,Washington,53061.0,1,0


## LA County

In [5]:
df_LA_county = df_counties.loc[df_counties['county']=='Los Angeles']

df_LA_county.tail()

,date,county,state,fips,cases,deaths
298102,2020-07-05,Los Angeles,California,6037.0,114993,3487
301186,2020-07-06,Los Angeles,California,6037.0,116570,3534
304274,2020-07-07,Los Angeles,California,6037.0,120539,3579
307364,2020-07-08,Los Angeles,California,6037.0,123004,3642
310459,2020-07-09,Los Angeles,California,6037.0,124738,3689


In [6]:
df_LA_county['date'] = pd.to_datetime(df_LA_county['date'])

df_LA_county.tail()

/Users/work/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,date,county,state,fips,cases,deaths
298102,2020-07-05,Los Angeles,California,6037.0,114993,3487
301186,2020-07-06,Los Angeles,California,6037.0,116570,3534
304274,2020-07-07,Los Angeles,California,6037.0,120539,3579
307364,2020-07-08,Los Angeles,California,6037.0,123004,3642
310459,2020-07-09,Los Angeles,California,6037.0,124738,3689


### Total cases and deaths over time

In [7]:
p = bokeh.plotting.figure(
    frame_height=300,
    frame_width=600,
    title="LA County",
    x_axis_type="datetime",
    x_axis_label="Date",
    y_axis_label="Total cases",
)

p.line(
    source=df_LA_county,
    x='date',
    y='cases',
    line_width=2,
)

p.yaxis[0].formatter = bokeh.models.formatters.BasicTickFormatter(use_scientific=False)

# Lab re-opening

lab_re_opening = bokeh.models.Span(
    location = pd.to_datetime("2020-06-08"),
    dimension='height',
    line_color='black',
    line_dash='dashed',
    line_width=2,
)

re_opening_label = bokeh.models.Label(
    x=pd.to_datetime("2020-06-08") - pd.DateOffset(days=35),
    y=25,
    y_units='screen',
    text='lab re-opening')

p.add_layout(lab_re_opening)
p.add_layout(re_opening_label)

bokeh.io.show(p)

In [8]:
p = bokeh.plotting.figure(
    frame_height=300,
    frame_width=600,
    title="LA County",
    x_axis_type="datetime",
    x_axis_label="Date",
    y_axis_label="Total deaths",
)

p.line(
    source=df_LA_county,
    x='date',
    y='deaths',
    color='orange',
    line_width=2,
)

# Lab re-opening

lab_re_opening = bokeh.models.Span(
    location = pd.to_datetime("2020-06-08"),
    dimension='height',
    line_color='black',
    line_dash='dashed',
    line_width=2,
)

re_opening_label = bokeh.models.Label(
    x=pd.to_datetime("2020-06-08") - pd.DateOffset(days=35),
    y=25,
    y_units='screen',
    text='lab re-opening')

p.add_layout(lab_re_opening)
p.add_layout(re_opening_label)

bokeh.io.show(p)

### New cases per day over time

In [9]:
# Two weeks ago from today
two_weeks_ago = yesterday - pd.DateOffset(days=13)

In [10]:
df_LA_county.head()

,date,county,state,fips,cases,deaths
9,2020-01-26,Los Angeles,California,6037.0,1,0
14,2020-01-27,Los Angeles,California,6037.0,1,0
19,2020-01-28,Los Angeles,California,6037.0,1,0
24,2020-01-29,Los Angeles,California,6037.0,1,0
29,2020-01-30,Los Angeles,California,6037.0,1,0


In [11]:
cases_array = np.array(df_LA_county["cases"])
new_cases_array = np.empty(np.shape(cases_array))

for i, n in enumerate(cases_array):
    if i > 0:
        new_cases_array[i] = cases_array[i] - cases_array[i-1]
        
df_LA_county["new cases"] = new_cases_array

df_LA_county.tail()

/Users/work/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,date,county,state,fips,cases,deaths,new cases
298102,2020-07-05,Los Angeles,California,6037.0,114993,3487,1496.0
301186,2020-07-06,Los Angeles,California,6037.0,116570,3534,1577.0
304274,2020-07-07,Los Angeles,California,6037.0,120539,3579,3969.0
307364,2020-07-08,Los Angeles,California,6037.0,123004,3642,2465.0
310459,2020-07-09,Los Angeles,California,6037.0,124738,3689,1734.0


In [12]:
p = bokeh.plotting.figure(
    frame_height=300,
    frame_width=600,
    title="LA County",
    x_axis_type="datetime",
    x_axis_label="Date",
    y_axis_label="New cases per day",
)

p.line(
    source=df_LA_county,
    x='date',
    y='new cases',
    line_width=2,
)

p.yaxis[0].formatter = bokeh.models.formatters.BasicTickFormatter(use_scientific=False)

bokeh.io.show(p)

In [13]:
p = bokeh.plotting.figure(
    frame_height=300,
    frame_width=600,
    title="LA County, 14 day trend",
    x_axis_type="datetime",
    x_axis_label="Date",
    y_axis_label="New cases per day",
)

p.line(
    source=df_LA_county,
    x='date',
    y='new cases',
    line_width=2,
)

p.x_range.start = two_weeks_ago
p.x_range.end = yesterday

p.yaxis[0].formatter = bokeh.models.formatters.BasicTickFormatter(use_scientific=False)

bokeh.io.show(p)

#### Seven-day moving average

In [14]:
df_LA_county['new cases (7 day average)'] = df_LA_county['new cases'].rolling(window=7).mean()

df_LA_county.tail()

/Users/work/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,date,county,state,fips,cases,deaths,new cases,new cases (7 day average)
298102,2020-07-05,Los Angeles,California,6037.0,114993,3487,1496.0,2442.714286
301186,2020-07-06,Los Angeles,California,6037.0,116570,3534,1577.0,2256.857143
304274,2020-07-07,Los Angeles,California,6037.0,120539,3579,3969.0,2430.000000
307364,2020-07-08,Los Angeles,California,6037.0,123004,3642,2465.0,2499.571429
310459,2020-07-09,Los Angeles,California,6037.0,124738,3689,1734.0,2438.714286


In [15]:
p = bokeh.plotting.figure(
    frame_height=300,
    frame_width=600,
    title="LA County",
    x_axis_type="datetime",
    x_axis_label="Date",
    y_axis_label="New cases per day (7 day average)",
)

p.line(
    source=df_LA_county,
    x='date',
    y='new cases (7 day average)',
    line_width=2,
)

p.yaxis[0].formatter = bokeh.models.formatters.BasicTickFormatter(use_scientific=False)

# Lab re-opening

lab_re_opening = bokeh.models.Span(
    location = pd.to_datetime("2020-06-08"),
    dimension='height',
    line_color='black',
    line_dash='dashed',
    line_width=2,
)

re_opening_label = bokeh.models.Label(
    x=pd.to_datetime("2020-06-08") - pd.DateOffset(days=35),
    y=25,
    y_units='screen',
    text='lab re-opening')

p.add_layout(lab_re_opening)
p.add_layout(re_opening_label)

bokeh.io.show(p)

In [16]:
p = bokeh.plotting.figure(
    frame_height=300,
    frame_width=600,
    title="LA County, 14 day trend",
    x_axis_type="datetime",
    x_axis_label="Date",
    y_axis_label="New cases per day (7 day average)",
)

p.line(
    source=df_LA_county, x="date", y="new cases (7 day average)", line_width=2,
)

p.x_range.start = two_weeks_ago
p.x_range.end = yesterday

p.yaxis[0].formatter = bokeh.models.formatters.BasicTickFormatter(use_scientific=False)

bokeh.io.show(p)

print("New cases (7-day average):")
new_cases = float(
    df_LA_county.loc[df_LA_county["date"] == yesterday]["new cases (7 day average)"]
)
print(f"Yesterday ({yesterday:%Y-%m-%d}): {new_cases:.1f}")
new_cases = float(
    df_LA_county.loc[df_LA_county["date"] == two_weeks_ago]["new cases (7 day average)"]
)
print(f"Two weeks ago ({two_weeks_ago:%Y-%m-%d}): {new_cases:.1f}")

New cases (7-day average):
Yesterday (2020-07-09): 2438.7
Two weeks ago (2020-06-26): 1946.1
